In [1]:
import pandas as pd

ratings_df = pd.read_csv("./ml-25m/ratings.csv")

In [2]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [3]:
ratings_df.shape

(25000095, 4)

In [4]:
# Check for missing values
missing_values = ratings_df.isnull().sum()
print("Missing values:")
print(missing_values)

Missing values:
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


In [5]:
uniq_users = pd.unique(ratings_df["userId"])
len(uniq_users)

162541

In [6]:
movie_users = pd.unique(ratings_df["movieId"])
len(movie_users)

59047

In [7]:
from sklearn.model_selection import train_test_split

# Initialize lists to store training and testing data
training_data = []
validation_data = []
testing_data = []

# Group ratings by user id
grouped_ratings = ratings_df.groupby('userId')

for _, group in grouped_ratings:
    # Split the ratings for each user into training, validation, and testing sets
    train_val, test = train_test_split(group, test_size=0.1, random_state=42)
    train, val = train_test_split(train_val, test_size=0.1, random_state=42)
    
    # Append training, validation, and testing data to respective lists
    training_data.append(train)
    validation_data.append(val)
    testing_data.append(test)

# Concatenate training, validation, and testing data for all users
training_df = pd.concat(training_data)
validation_df = pd.concat(validation_data)
testing_df = pd.concat(testing_data)

# Save training, validation, and testing data to CSV files
training_df.to_csv("training.csv", index=False)
validation_df.to_csv("validation.csv", index=False)
testing_df.to_csv("testing.csv", index=False)

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Collaborative Filtering Example") \
    .getOrCreate()

# Load training data
training_data = spark.read.csv("training.csv", header=True, inferSchema=True)

# Initialize ALS model
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")

# Fit the model to the training data
model = als.fit(training_data)

# Evaluate the model on validation data
validation_data = spark.read.csv("validation.csv", header=True, inferSchema=True)
predictions = model.transform(validation_data)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

print("Root Mean Squared Error (RMSE) on validation data = " + str(rmse))

# Stop SparkSession
spark.stop()


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [2]:
!python —m ipykernel install --user --name PYSPARK_KERNEL

python: can't open file 'C:\\Users\\somes\\OneDrive\\Documents\\GitHub\\Advanced-Data-Mining\\Assignment 2\\—m': [Errno 2] No such file or directory
